In [16]:
#Importing the necessary libraries
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import os
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, InputLayer

In [17]:
# Function to load and preprocess images
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array_expanded_dims)

In [18]:
# Load the Dataset 
data_path = 'Example Data-20240208T214429Z-001/Example Data/ImageLabels.xlsx'
images_folder = 'Example Data-20240208T214429Z-001/Example Data/exported'
df = pd.read_excel(data_path)

In [19]:
# Preprocess the DataFrame
df['Aliases'].fillna('None', inplace=True)
df['Parents'].fillna('None', inplace=True)
df['preprocessed_image'] = df['Image Name'].apply(lambda x: load_and_preprocess_image(os.path.join(images_folder, x)))

C:\Users\Hussein\AppData\Local\Temp\ipykernel_14588\1882043891.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Aliases'].fillna('None', inplace=True)
C:\Users\Hussein\AppData\Local\Temp\ipykernel_14588\1882043891.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

In [20]:
# Encode labels
label_encoder = LabelEncoder()
df['label_encoded'] = label_encoder.fit_transform(df['Label'])
num_classes = df['label_encoded'].nunique()
y = to_categorical(df['label_encoded'], num_classes=num_classes)

In [21]:
# Prepare the dataset
X = np.array(df['preprocessed_image'].tolist())
y = df['label_encoded'].values
weights = df['Confidence'] / 100  # Normalize confidence scores to be between 0 and 1

In [22]:
# Split dataset
X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X, y, weights, test_size=0.2, random_state=42)

In [23]:
y_train = to_categorical(y_train, num_classes)


In [24]:
# Define the CNN model
model = Sequential([
    InputLayer(input_shape=(224, 224, 3)),  # Adjust input shape based on  image preprocessing
    Conv2D(32, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # 'num_classes' should match the number of categories
])

c:\Python312\Lib\site-packages\keras\src\layers\core\input_layer.py:25: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


In [25]:
#compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
# Ensure X_train is a numpy array and has the correct shape
X_train = np.squeeze(X_train)

# Check the shape of X_train
print("Shape of X_train:", X_train.shape)

Shape of X_train: (318, 224, 224, 3)


In [27]:

model.fit(X_train, y_train, sample_weight=weights_train, epochs=50, validation_split=0.1)


Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 433ms/step - accuracy: 0.0204 - loss: 690.6006 - val_accuracy: 0.0000e+00 - val_loss: 4.6086
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 432ms/step - accuracy: 0.0299 - loss: 4.6640 - val_accuracy: 0.0312 - val_loss: 4.6396
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 416ms/step - accuracy: 0.0377 - loss: 4.6411 - val_accuracy: 0.0312 - val_loss: 4.6485
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 403ms/step - accuracy: 0.0570 - loss: 4.6376 - val_accuracy: 0.0312 - val_loss: 4.6454
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 401ms/step - accuracy: 0.0465 - loss: 4.6595 - val_accuracy: 0.0312 - val_loss: 4.6413
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 403ms/step - accuracy: 0.0465 - loss: 4.6644 - val_accuracy: 0.0312 - val_loss: 4.6381
Epoch 7/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 396ms/step - accuracy: 0.0381 - loss: 4.6521 - val_accuracy: 0.0312 - val_loss: 4.6349
Epoch 8/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 393ms/step - accuracy: 0.0381 - loss: 4.5963 - val_accuracy: 0.0312 - val